### 示例：使用朴素贝叶斯分类器从个人广告中获取区域倾向
（1）收集数据：从RSS(Really Simple Syndication（简易信息聚合）)源收集内容，这里需要对RSS源构建一个接口。  
（2）准备数据：将文本文件解析成词条向量。  
（3）分析数据：检查词条确保解析的正确性。  
（4）训练算法：使用我们之前建立的trainNB0()函数。  
（5）测试算法：观察错误率，确保分类器可用。可以修改切分程序，以降低错误率，提高分类结果。  
（6）使用算法：构建一个完整的程序，封装所有内容。给定两个RSS源，该程序会显示最常用的公共词。  

In [1]:
def localWords(feed1,feed0):
    import feedparser
    docList=[]; classList=[]; fullText=[]
    minLen = min(len(feed1['entries']),len(feed0['entries']))  #取feed0和feed1中较小的长度
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])  #读取每一条页面返回的summary，并将字符串转换成字符串列表
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) #标记属于第一类网站
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)  #标记属于第二类网站
    vocabList = createVocabList(docList)  #创建词汇表，不重复
    top30Words = calcMostFreq(vocabList,fullText)  #找到词频排在前30的词
    # 说明：
    # 1.为什么要去除频次最高的词项：因为出现频次高的词项很多是没有意义的冗余和辅助性内容，
    # 不移除这些词汇，使用朴素贝叶斯分类算法，其错误率会显著高一些
    # - 另外一个方法，是从词表中删除辅助词，这个需要删除的词表被称为停用词表，
    # 这个网站列出了多门语言，如英文、中文等语言中的停用词表。
    for pairW in top30Words:     #去掉词频排在前30的词
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = list(range(2*minLen)); testSet=[]
    for i in range(20):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0.0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList,docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is:', float(errorCount)/len(testSet))
    return vocabList,p0V,p1V

def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V = localWords(ny,sf)
    topNY = []; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0: topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0: topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair:pair[1], reverse=True)
    print('SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**')
    for item in sortedSF:
        print(item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print('NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**')
    for item in sortedNY:
        print(item[0])